In [ ]:
import pandas as pd # importar pandas
import numpy as np
import matplotlib.pyplot as plt


###Comenzamos con la base de datos en crudo, para posteriormente hacer una limpieza de los datos.
###Recordemos que la base de datos la obtuvimos a partir del (EM-DAT) The International Disaster Data Base.

#### Preguntas que buscamos resolver
Frecuencia e impacto
    • ¿Qué tipos de desastres son los más frecuentes en el mundo?
    • ¿Qué desastres causan más muertes?
    • ¿Qué desastres generan mayores pérdidas económicas?
Vulnerabilidad por países
    • ¿Qué países sufren más desastres?
    • ¿Qué regiones requieren mayor presupuesto de ayuda?
Respuesta internacional
    • ¿Qué países reciben más ayuda internacional?
    • ¿Qué países tienen los menores tiempos de respuesta?
    • ¿Qué tan eficiente es la ayuda enviada?
Análisis relacional
    • ¿Qué relación existe entre la severidad y la mortalidad?
    • ¿El tiempo de respuesta influye en el número de fallecidos?
    • ¿La ayuda económica acelera la recuperación?

In [ ]:
df_raw = pd.read_excel("public_emdat_2025-12-09.xlsx") # leer el archivo Excel

df_raw.head()



###Osevamos los nombres de las 46 columnas que posteriormente podrémos utilizar para el análisis de datos.


In [ ]:
df_raw.columns

In [ ]:
df_raw.info()

###También observamos que los desastres se dividen en Neturales y Tecnológicos, y decidimos enfocarnos más en los desastres Naturales y eliminar los tecnológicos y extraemos las primeras 5 filas.

In [ ]:
df_naturales = df_raw[df_raw["Disaster Group"] == "Natural"].copy()

In [ ]:
df_naturales.head()

In [ ]:
df_naturales.info()

### Revisamos si hay duplicados, pero como la base viene de un repositorio oficial muy serio, nos damos cuenta en que no los hay.


In [ ]:
df_naturales["DisNo."].duplicated().sum()

### A partir de las 46 columnas de la bases de datos decidimos eliminas columnas que no van a ser utiles en el Análisis de datos.

In [ ]:
columnas_a_quitar = ["Historic","Classification Key", "External IDs", "Disaster Group","Event Name","ISO","Origin","Associated Types","OFDA/BHA Response","Declaration","Appeal","River Basin","Start Day","End Year","End Month","End Day","No. Injured","No. Affected","No. Homeless","Reconstruction Costs ('000 US$)","Reconstruction Costs, Adjusted ('000 US$)","Insured Damage ('000 US$)","Insured Damage, Adjusted ('000 US$)","CPI","Admin Units","Entry Date","Last Update"]
df_naturales.drop(columns=columnas_a_quitar, inplace=True) # eliminamos columnas que no usaremos
df_naturales.info() # revisamos nuevamente la información del DataFrame
df_naturales.columns # revisamos los nombres de las columnas restantes

### Guardamos el dataframe limpio como CSV, para uso futuro


In [ ]:
df_naturales.to_csv("desastres_naturales_clean.csv", index=False)


### Iniciar rapidamente un análisis exploratorio de los paises con más desastres naturales y obtenemos que China , USA y la India son los paises con mayor cantidad de desastres naturales.  


In [ ]:
# iniciar rapidamente un análisis exploratorio de los paises con más desastres naturales
desastres_por_pais = df_naturales["Country"].value_counts() # value_counts para contar ocurrencias por evento
top_15_paises = desastres_por_pais.head(15) # head para obtener los 15 primeros
print(top_15_paises)

### Consideramos descartar los países que no tengan como menos 20 desastres naturales.
### Obtenemos como resultado que de los 227 paises originales solo 121 tienen al menos 20 desastres registrados y realizamos una grafica.


In [ ]:
paises_confiables= desastres_por_pais[desastres_por_pais >= 20].index.tolist()

print(len(paises_confiables))

In [ ]:
# Gráfica de desastres por pais
plt.figure(figsize=(12,6)) # ajustar tamaño de la figura 12 de ancho por 6 de alto
top_15_paises.plot(kind='bar') 
plt.title("Top 15 países con más desastres naturales (1900-2024)")
plt.xlabel("País")
plt.ylabel("Número de desastres")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Sumamos el numero de desastre por tipo.
### EN la grafica podemos observar que son màs comunes las hinundaciones.
### Existen al rededor de 4000 hinundaciones en nuestra base de datos.

In [ ]:
df_naturales
stats_por_tipo = (
    df_naturales
    .groupby("Disaster Type")
    .agg(
        num_eventos = ("DisNo.", "nunique"),         # cuántos desastres únicos
        filas_registro = ("Disaster Type", "size"),  # cuántas filas (por si un desastre se repite)
        muertes_totales = ("Total Deaths", "sum"),
        muertes_promedio = ("Total Deaths", "mean"),
        danio_total_kusd = ("Total Damage, Adjusted ('000 US$)", "sum"),
        danio_promedio_kusd = ("Total Damage, Adjusted ('000 US$)", "mean"),
    )
    .sort_values("num_eventos", ascending=False)
)

stats_por_tipo

### 1. ¿Qué tipos de desastres son los más frecuentes en el mundo?
#### POdemos observar que las hinundaciones son de todos el desastre màs frecuente.

In [ ]:
stats_por_tipo.to_csv("estadisticas_desastres_por_tipo.csv")
#graficar stats por tipo de desastre
plt.figure(figsize=(12,6))
stats_por_tipo["num_eventos"].plot(kind='bar')
plt.title("Número de eventos por tipo de desastre natural")
plt.xlabel("Tipo de desastre")
plt.ylabel("Número de eventos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2.¿Qué desastres causan más muertes?
#### 3 ¿Qué desastres generan mayores pérdidas económicas?

In [ ]:
tabla_muertes = (
    .groupby("Disaster Subtype")
    .agg(
        muertes_totales=("Total Deaths", "sum"),
        afectados_totales=("Total Affected", "sum"),
    )
    .reset_index()
)
tabla_muertes

### Extraemos sólo las inundaciones 

In [ ]:
df_inundaciones = df_naturales[df_naturales["Disaster Type"] == "FLood"].copy()
df_inundaciones.head()

In [ ]:
tabla_inundaciones = (
    df_inundaciones.groupby("Disaster Subtype")
    .agg(
        muertes_totales=("Total Deaths", "sum"),
        afectados_totales=("Total Affected", "sum"),
    )
    .reset_index()
)
tabla_inundaciones

### Extraemos sólo las tormentas.

In [ ]:
df_tormenta = df_naturales[df_naturales["Disaster Type"] == "Storm"].copy()
df_tormenta.head()

### Extraemos sólo los Terremotos.

In [ ]:
df_terremotos = df_naturales[df_naturales["Disaster Type"] == "Earthquake"].copy()
df_terremotos.head()

### Extraemos el numero de sismos por año del año 2000 al 2025

In [ ]:
sismos_por_anios = df_terremotos["Start Year"].value_counts().sort_index()
sismos_por_anios

In [ ]:
tabla_terremotos = (
    df_inundaciones.groupby("Disaster Subtype")
    .agg(
        muertes_totales=("Total Deaths", "sum"),
        afectados_totales=("Total Affected", "sum"),
    )
    .reset_index()
)
tabla_terremotos

In [ ]:
tabla_inundaciones
tabla_terremotos

### Extraemos sólo los Huracanes 

In [ ]:
df_huracanes = df_naturales[df_naturales["Disaster Type"] == "Storm"].copy()
df_huracanes.head()

### Extraemos sólo las pandemias.

In [ ]:
df_pandemias = df_naturales[df_naturales["Disaster Type"] == "Epidemic"].copy()
df_pandemias.head()

### Extraemos los subtipos de pandemias las cuales son Bacterianas, de Parásicas, Infexiosas y Virales.

### Para posterirmente saber cuales son las pandemias más mortales

In [ ]:
tipo_pandemia=df_pandemias["Disaster Subtype"].unique() 
tipo_pandemia

### Obtenemos las muertes totales y el total de afectados a causa de pandemias.

### Las enfermedades que se pueden expandir como una pandemia  se clasifican según su agente causal:
#### Bacterianas (tuberculosis, cólera), Infecciosas, Virales (gripe, COVID-19, sarampión) y  Parasitarias (malaria, amebiasis).

In [ ]:
tabla_pandemias = (
    df_pandemias.groupby("Disaster Subtype")
    .agg(
        muertes_totales=("Total Deaths", "sum"),
        afectados_totales=("Total Affected", "sum"),
    )
    .reset_index()
)
tabla_pandemias

In [ ]:
# Gráfica por tipo de pandemia 
plt.figure(figsize=(10,5))
tipo_pandemia(kind='bar') 
plt.title("Tipos de enfermedades")
plt.xlabel("Pandemias")
plt.ylabel("Tipo de enfermedades ")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()